In [5]:
import os
import pandas as pd
import tensorflow as tf

In [77]:
os.chdir("C:\\Users\\yana1\\Downloads")

In [79]:
data = pd.read_csv('thesisdata_cleaned_fullsample.csv')

data = data.drop(columns = ['Unnamed: 0'])
data.head()

,type,gender_guessed,Height,Width,is_flat,age,years_selling,Uniqueness,is_signed,gallery,price_numeric,painting,description_present,location,image_path
0,Unique work,female,27.6,27.6,Not flat,0.0,1.0,Unique,signed,independent,3158.0,other,Present,France,images_thesis\2313827_1_m.jpg
1,Unique work,female,19.7,19.7,Not flat,0.0,3.0,Unique,signed,independent,2065.0,other,Present,France,images_thesis\2328028_1_m.jpg
2,Unique work,female,23.6,23.6,Not flat,0.0,3.0,Unique,signed,independent,3522.0,other,Present,France,images_thesis\2299335_1_m.jpg
3,Unique work,female,15.7,11.8,Not flat,4.0,5.0,Unique,signed,gallery,644.0,oil,Present,other,images_thesis\1118562_1_m.jpg
4,Unique work,unknown,15.7,15.7,Not flat,4.0,5.0,Unique,signed,independent,838.0,other,Present,France,images_thesis\1241121_1_m.jpg


In [81]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20917 entries, 0 to 20916
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   type                 20917 non-null  object 
 1   gender_guessed       20917 non-null  object 
 2   Height               20917 non-null  float64
 3   Width                20917 non-null  float64
 4   is_flat              20917 non-null  object 
 5   age                  20917 non-null  float64
 6   years_selling        20917 non-null  float64
 7   Uniqueness           20917 non-null  object 
 8   is_signed            20917 non-null  object 
 9   gallery              20917 non-null  object 
 10  price_numeric        20917 non-null  float64
 11  painting             20917 non-null  object 
 12  description_present  20917 non-null  object 
 13  location             20917 non-null  object 
 14  image_path           20917 non-null  object 
dtypes: float64(5), object(10)
memory usa

In [83]:
numeric = data.select_dtypes(include = 'number').columns.drop('price_numeric')
categorical = data.select_dtypes(include = 'object').drop(columns = ['image_path']).columns

print(numeric)
print(categorical)

Index(['Height', 'Width', 'age', 'years_selling'], dtype='object')
Index(['type', 'gender_guessed', 'is_flat', 'Uniqueness', 'is_signed',
       'gallery', 'painting', 'description_present', 'location'],
      dtype='object')


In [85]:

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(data[numeric].to_numpy())   # learn mean + variance

string_lookups = {}

for col in categorical:
    lookup = tf.keras.layers.StringLookup(output_mode="int")
    lookup.adapt(data[col])    # learn unique categories
    string_lookups[col] = lookup

one_hot_encoders = {}

for col in categorical:
    encoder = tf.keras.layers.CategoryEncoding(
        num_tokens=string_lookups[col].vocabulary_size(),
        output_mode="one_hot"
    )
    one_hot_encoders[col] = encoder


In [87]:
# preprocess, build the model

encoded_features = []

numeric_data = tf.keras.layers.Concatenate(axis=-1)(
    [inputs[col] for col in numeric]
)
encoded_numeric = normalizer(numeric_data)
encoded_features.append(encoded_numeric)


for col in categorical:
    int_encoded = string_lookups[col](inputs[col])       
    one_hot = one_hot_encoders[col](int_encoded)         
    encoded_features.append(one_hot)

x = tf.keras.layers.concatenate(encoded_features)

# 4.4 — Build the dense neural network
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer="adam", loss="mse")


In [89]:
#test and debug

sample = data.sample(100, random_state=42).copy()


X_debug = {}

for col in numeric:
    X_debug[col] = sample[col].values

for col in categorical:
    X_debug[col] = sample[col].values


In [91]:
X_debug

{'Height': array([20.  , 14.6 , 39.4 , 12.  , 21.7 , 15.7 , 13.8 , 36.2 , 40.6 ,
        18.9 , 41.3 , 27.6 , 39.4 , 63.8 , 35.4 , 10.2 , 19.7 , 22.6 ,
        22.4 , 23.6 , 31.5 , 55.9 , 27.  , 55.1 , 31.5 , 44.9 , 29.  ,
        22.  , 43.3 , 17.75, 43.3 , 19.7 , 47.6 , 30.  , 17.7 , 15.7 ,
        10.2 , 27.6 ,  7.1 , 31.5 , 16.1 , 23.6 , 32.  , 23.6 , 19.7 ,
        22.8 , 24.4 ,  9.4 ,  0.4 , 55.1 , 23.6 , 10.6 , 25.6 , 19.7 ,
        19.  , 47.2 , 35.  , 31.5 , 31.9 , 23.6 , 39.4 , 39.4 , 19.7 ,
        19.7 ,  7.9 , 19.7 , 55.1 , 19.7 , 15.7 , 31.5 , 19.7 ,  9.8 ,
        23.6 , 19.7 , 17.7 , 31.5 , 31.5 , 13.  , 37.6 , 23.6 , 30.7 ,
        23.6 , 31.5 , 15.9 , 35.4 , 23.6 ,  7.1 , 27.6 , 19.6 , 27.6 ,
        13.  , 30.  , 51.2 , 39.4 , 35.4 , 39.4 , 21.7 , 35.  , 40.2 ,
        29.9 ]),
 'Width': array([24.  , 25.2 , 39.4 ,  8.  , 13.  , 15.7 , 19.7 , 28.7 , 28.7 ,
        24.4 , 61.  , 39.4 , 51.2 , 51.2 , 35.4 , 14.2 , 15.7 , 29.9 ,
        29.9 , 23.6 , 23.6 , 39.8 , 24.  

In [93]:
# output vector
y_debug = sample["price_numeric"].values


In [95]:
# train on sample
history = model.fit(
    X_debug,
    y_debug,
    epochs=5,
    batch_size=16,
    verbose=1
)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 24608020.0000
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24602184.0000 
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24594642.0000 
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24583696.0000 
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24565816.0000 


In [105]:
def load_image(path, img_size=(224, 224)):
    img = tf.io.read_file(path)  # read bytes
    img = tf.image.decode_image(img, channels=3, expand_animations=False)  # decode JPEG/PNG
    img = tf.image.resize(img, img_size)  # resize
    img = tf.cast(img, tf.float32)  # convert to float (0–255 range)
    return img


In [107]:
import tensorflow as tf

# 1. Input layer
image_input = tf.keras.Input(shape=(224, 224, 3), name="image")

# 2. Normalize pixels 0–255 → 0–1
x = tf.keras.layers.Rescaling(1/255.0)(image_input)

# 3. Convolutional layers
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

# 4. Flatten for dense layers
x = tf.keras.layers.Flatten()(x)

# 5. Dense layers
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

# 6. Output layer (regression)
output = tf.keras.layers.Dense(1)(x)

# 7. Build and compile
image_model = tf.keras.Model(inputs=image_input, outputs=output)
image_model.compile(optimizer='adam', loss='mse')


In [109]:
sample = data.iloc[:100].copy()


In [111]:
X_images = tf.convert_to_tensor([load_image(p) for p in sample["image_path"]])


In [113]:
X_images.shape

TensorShape([100, 224, 224, 3])

In [115]:
y_images = sample["price_numeric"].values
print(y_images.shape)


(100,)


In [117]:
history = image_model.fit(
    X_images,
    y_images,
    epochs=5,      # small number to debug
    batch_size=16,
    verbose=1
)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 317ms/step - loss: 2166356992.0000
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 305ms/step - loss: 2123328896.0000
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 301ms/step - loss: 2093345280.0000
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - loss: 2066524288.0000
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 297ms/step - loss: 2070920576.0000


In [119]:
import tensorflow as tf

# ============================================================
# 1) TABULAR INPUTS + PREPROCESSING
# ============================================================

# Create Input layers for tabular data
inputs = {}

# Numeric inputs
for col in numeric:
    inputs[col] = tf.keras.Input(shape=(1,), name=col)

# Categorical inputs
for col in categorical:
    inputs[col] = tf.keras.Input(shape=(1,), dtype=tf.string, name=col)

encoded_features = []

# --- Numeric preprocessing ---
numeric_inputs = [inputs[col] for col in numeric]
numeric_concat = tf.keras.layers.Concatenate()(numeric_inputs)
numeric_encoded = normalizer(numeric_concat)
encoded_features.append(numeric_encoded)

# --- Categorical preprocessing ---
for col in categorical:
    int_encoded = string_lookups[col](inputs[col])
    one_hot = one_hot_encoders[col](int_encoded)
    encoded_features.append(one_hot)

# Tabular feature vector
tabular_features = tf.keras.layers.Concatenate()(encoded_features)


# ============================================================
# 2) IMAGE INPUT + CNN BRANCH
# ============================================================

image_input = tf.keras.Input(shape=(224, 224, 3), name="image")

i = tf.keras.layers.Rescaling(1/255.0)(image_input)

i = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(i)
i = tf.keras.layers.MaxPooling2D()(i)

i = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(i)
i = tf.keras.layers.MaxPooling2D()(i)

i = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(i)
i = tf.keras.layers.MaxPooling2D()(i)

i = tf.keras.layers.Flatten()(i)
i = tf.keras.layers.Dense(128, activation='relu')(i)
i = tf.keras.layers.Dense(64, activation='relu')(i)

image_features = i


# ============================================================
# 3) COMBINE TABULAR + IMAGE BRANCHES
# ============================================================

combined = tf.keras.layers.Concatenate()([tabular_features, image_features])

x = tf.keras.layers.Dense(128, activation='relu')(combined)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)


# ============================================================
# 4) BUILD THE MULTIMODAL MODEL
# ============================================================

all_inputs = list(inputs.values()) + [image_input]

multimodal_model = tf.keras.Model(inputs=all_inputs, outputs=output)

multimodal_model.compile(
    optimizer='adam',
    loss='mse'
)

multimodal_model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)            │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ image[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 222, 222, 32)      │             896 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 111, 111, 32)      │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 109, 109, 64)      │          18,496 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 54, 54, 64)        │               0 │ conv2d_4[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 52, 52, 128)       │          73,856 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Height (InputLayer)           │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Width (InputLayer)            │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ age (InputLayer)              │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ years_selling (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ type (InputLayer)             │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gender_guessed (InputLayer)   │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ is_flat (InputLayer)          │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Uniqueness (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ is_signed (InputLayer)        │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 11,201,930 (42.73 MB)

 Trainable params: 11,201,921 (42.73 MB)

 Non-trainable params: 9 (40.00 B)

In [123]:
sample = data.iloc[:100].copy()
sample.head()

,type,gender_guessed,Height,Width,is_flat,age,years_selling,Uniqueness,is_signed,gallery,price_numeric,painting,description_present,location,image_path
0,Unique work,female,27.6,27.6,Not flat,0.0,1.0,Unique,signed,independent,3158.0,other,Present,France,images_thesis\2313827_1_m.jpg
1,Unique work,female,19.7,19.7,Not flat,0.0,3.0,Unique,signed,independent,2065.0,other,Present,France,images_thesis\2328028_1_m.jpg
2,Unique work,female,23.6,23.6,Not flat,0.0,3.0,Unique,signed,independent,3522.0,other,Present,France,images_thesis\2299335_1_m.jpg
3,Unique work,female,15.7,11.8,Not flat,4.0,5.0,Unique,signed,gallery,644.0,oil,Present,other,images_thesis\1118562_1_m.jpg
4,Unique work,unknown,15.7,15.7,Not flat,4.0,5.0,Unique,signed,independent,838.0,other,Present,France,images_thesis\1241121_1_m.jpg


In [125]:
X_multi = {}

# numeric
for col in numeric:
    X_multi[col] = sample[col].values

# categorical
for col in categorical:
    X_multi[col] = sample[col].values


In [127]:
X_images = tf.convert_to_tensor([
    load_image(p) for p in sample["image_path"]
])


In [129]:
X_images.shape

TensorShape([100, 224, 224, 3])

In [137]:
X_multi["image"] = X_images
X_multi


{'Height': array([27.6, 19.7, 23.6, 15.7, 15.7, 31.5, 19.7, 19.3, 47.2, 31.9,  8. ,
        31.9, 31.5, 28.7, 31.5, 59.1,  7.9, 14.6, 35.4, 40. , 63.8, 27.6,
        24. , 63. , 47.2, 48. , 36. , 31.9, 25.6, 16.5, 31.5, 31.9, 35.4,
        12.6, 19.7, 23.6, 15.7, 29.5, 23.6, 16.5, 23.6, 27.6, 39.4,  7.9,
        21.7, 39.4, 45.7, 11.8, 15.7, 47.2, 21.7, 39.4, 28.7, 28.7,  9.8,
        48. , 15.7, 31.5, 31.5, 28.7, 27.6, 28.7, 23.6, 47.2, 15.7, 16. ,
        47.2, 36. , 15.7, 14. , 12. , 41.3, 31.5, 31.5, 39.4, 22. , 25.2,
        15.4, 27.6, 27.6, 19.7, 18.1, 31.5, 29.5, 31.5, 46.1, 35.4, 16.5,
        39.4, 35.4, 35.4, 31.5, 19.7, 47.2, 31.5,  6.9, 39.4, 39.4, 13. ,
        44.5]),
 'Width': array([27.6, 19.7, 23.6, 11.8, 15.7, 23.6, 19.7, 19.3, 31.5, 39.4,  8. ,
        46.1, 31.5, 21.3, 31.5, 78.7,  7.9, 10.6, 35.4, 66. , 38.2, 19.7,
        19.7, 37.4, 23.6, 30. , 24. , 39.4, 19.7, 11.7, 23.6, 39.4, 27.6,
        10.2,  7.9, 11.8, 11.8, 48. , 23.6, 11.8, 31.5, 19.7, 31.9,  7.9,
   

In [139]:
y_multi = sample["price_numeric"].values


In [141]:
multimodal_model.fit(
    X_multi,
    y_multi,
    epochs=5,
    batch_size=16
)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - loss: 2166754048.0000
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 295ms/step - loss: 2091914752.0000
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 299ms/step - loss: 2134672000.0000
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 306ms/step - loss: 2073891200.0000
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 300ms/step - loss: 2077431552.0000
